# Generating a list of Animeplanet usernames
* Looks at followers and following

In [ ]:
import os
import random
import re
import time

In [ ]:
PROXY_NUMBER = 0

In [ ]:
name = "get_friend_username"
source = "animeplanet"

In [ ]:
%run WebEndpointBase.ipynb

In [ ]:
# if we rerunning the notebook, then resume execution where we last left off
usernames = set()
if os.path.exists("friend_usernames.txt"):
    with open("friend_usernames.txt") as f:
        usernames = {x.strip() for x in f.readlines() if x.strip()}

recent = set()
if os.path.exists("character_usernames.txt"):
    with open("character_usernames.txt") as f:
        recent = {x.strip() for x in f.readlines() if x.strip()}

visited = set()
if os.path.exists("friend_visited.txt"):
    with open("friend_visited.txt") as f:
        visited = {x.strip() for x in f.readlines() if x.strip()}

logger.info(
    f"Starting with {len(usernames | recent)} stored usernames and {len(visited)} visited usernames"
)

In [ ]:
def order_users(usernames, recent, visited):
    order = list((usernames | recent) - visited)
    random.shuffle(order)
    return order

In [ ]:
def get_users_on_url(url):
    response = call_api(url)
    if response.status_code in [404]:
        logger.warning(f"Error {response} received when handling {url}")
        return set()
    if not response.ok:
        logger.warning(f"Error {response} received when handling {url}")
        return set()
    urls = re.findall("""/users/[^"/#%?.'><]+""", response.text)
    users = {x[len("/users/") :] for x in urls}
    return users

In [ ]:
def get_users_on_base_url(base_url):
    users = set()
    page = 1
    while True:
        url = base_url + f"?page={page}"
        new_users = get_users_on_url(url)
        if len(users) == len(users | new_users):
            break
        users |= new_users
        page += 1
    return users

In [ ]:
def get_users(user):
    users = [
        get_users_on_base_url(f"https://www.anime-planet.com/users/{user}/{x}")
        for x in ["followers", "following"]
    ]
    return set().union(*users)

In [ ]:
while True:
    order = order_users(usernames, recent, visited)
    for user in order:
        users = get_users(user)
        visited.add(user)
        usernames |= users
        if should_save("users", max_iters=500):
            atomic_to_csv(sorted(list(usernames)), "friend_usernames.txt")
            atomic_to_csv(sorted(list(visited)), "friend_visited.txt")
            logger.info(
                f"Successfully written {len(usernames)} users and visited {len(visited)} usernames"
            )
    time.sleep(1)